# Problem 93

By using each of the digits from the set, `{1, 2, 3, 4}`, exactly once, and making use of the four arithmetic operations `(+, −, *, /)` and brackets/parentheses, it is possible to form different positive integer targets.

For example,

```
8 = (4 * (1 + 3)) / 2
14 = 4 * (3 + 1 / 2)
19 = 4 * (2 + 3) − 1
36 = 3 * 4 * (2 + 1)
```

Note that concatenations of the digits, like `12 + 34`, are not allowed.

Using the set, `{1, 2, 3, 4}`, it is possible to obtain thirty-one different target numbers of which 36 is the maximum, and each of the numbers 1 to 28 can be obtained before encountering the first non-expressible number.

Find the set of four distinct digits, `a < b < c < d`, for which the longest set of consecutive positive integers, 1 to n, can be obtained, giving your answer as a string: abcd.

In [1]:
import operator
from collections import namedtuple, defaultdict
from itertools import combinations, product

In [2]:
digits = list(combinations(range(1,10), 4))

In [3]:
Operator = namedtuple("Operator", "op commutative")
operators = [
    Operator(operator.add, True),
    Operator(operator.sub, False),
    Operator(operator.mul, True),
    Operator(operator.truediv, False),
]

In [4]:
def all_operations(values) -> list:
    data = defaultdict(lambda: defaultdict(list))
    for v in values:
        data[1][frozenset({v})].append(v)
    
    for stage in range(2, len(values) + 1):
        keys = [k for k in data if k < stage]
        partial_operands = [(operands, value) for k in keys for operands, value in data[k].items()]
        for p_op in partial_operands:
            applicable = [p for p in partial_operands if not p[0] & p_op[0]]
            for p_app in applicable:
                
                new_operands = p_app[0].union(p_op[0])
                results = []
                for operator in operators:
                    for v1, v2 in product(p_app[1], p_op[1]):
                        try:
                            results.append(operator.op(v1, v2))
                        except ZeroDivisionError:
                            pass
                        
                        if not operator.commutative:
                            try:
                                results.append(operator.op(v2, v1))
                            except ZeroDivisionError:
                                pass

                data[len(new_operands)][new_operands].extend(results)
        
    all_results = [data[len(values)][k] for k in data[len(values)]]
    all_results = [r for results in all_results for r in results]
    all_results = {int(r) for r in all_results if r > 0 and r == int(r) }
    return all_results
    
def first_missing(all_results):
    all_numbers = set(range(1, len(all_results) + 1))
    return min(all_numbers - all_results)

In [5]:
max_missing = -1
max_digits = None

for d in digits:
    results = all_operations(d)
    f_missing = first_missing(results)
    if f_missing > max_missing:
        max_missing = f_missing
        max_digits = d
        
print(max_digits)

(1, 2, 5, 8)
